In [161]:
L = 100
W = 100
H = 100

id = [1, 2, 3, 4, 5]
l = [40, 50, 30, 20, 70]
w = [40, 50, 30, 60, 30]
h = [40, 20, 30, 20, 40]
N = len(id)

p = [1, 0, 0, 0, 1] #priority

In [162]:
from gurobipy import *

In [163]:
model_1 = Model("Base Model")

In [164]:
x = model_1.addVars(N, vtype = GRB.BINARY, name = "x")

X = model_1.addVars(N, vtype = GRB.CONTINUOUS, lb = 0)
Y = model_1.addVars(N, vtype = GRB.CONTINUOUS, lb = 0)
Z = model_1.addVars(N, vtype = GRB.CONTINUOUS, lb = 0)

a = model_1.addVars(N, N, vtype = GRB.BINARY)
b = model_1.addVars(N, N, vtype = GRB.BINARY)
c = model_1.addVars(N, N, vtype = GRB.BINARY)

In [165]:
# we use disjunctive constraints

model_1.addConstrs(X[i] + l[i] <= (1 - x[i])*L + L
                  for i in range(N))

model_1.addConstrs(Y[i] + w[i] <= (1 - x[i])*W + W
                  for i in range(N))

model_1.addConstrs(Z[i] + h[i] <= (1 - x[i])*H + H
                  for i in range(N))

model_1.addConstrs(a[i, i] == 0 for i in range(N))

model_1.addConstrs(b[i, i] == 0 for i in range(N))

model_1.addConstrs(c[i, i] == 0 for i in range(N))

model_1.addConstrs(a[i, j] + a[j, i] >= x[i] + x[j] - 1
                      for i in range(N) for j in range(N) if i != j)

model_1.addConstrs(b[i, j] + b[j, i] >= x[i] + x[j] - 1
                      for i in range(N) for j in range(N) if i != j)

model_1.addConstrs(c[i, j] + c[j, i] >= x[i] + x[j] - 1
                      for i in range(N) for j in range(N) if i != j)

model_1.addConstrs(a[i, j] + a[j, i] <= 1
                      for i in range(N) for j in range(N) if i != j)

model_1.addConstrs(b[i, j] + b[j, i] <= 1
                      for i in range(N) for j in range(N) if i != j)

model_1.addConstrs(c[i, j] + c[j, i] <= 1
                      for i in range(N) for j in range(N) if i != j)

model_1.addConstrs(X[i] + l[i] <= (1 - a[i, j])*L + X[j]
                  for i in range(N) for j in range(N))

model_1.addConstrs(Y[i] + w[i] <= (1 - b[i, j])*W + Y[j]
                  for i in range(N) for j in range(N))

model_1.addConstrs(X[i] + h[i] <= (1 - c[i, j])*H + Z[j]
                  for i in range(N) for j in range(N))


model_1.addConstrs(x[i] >= p[i]*(x[j] - p[j])
                  for i in range(N) for j in range(N))
#does not include the case if priority item is not feasible, but another items are feasible

#A better way to give priority is adding weights in objevtive function, N for priority ones and 1 for rest.

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,


In [166]:
ob_fn = sum(x[i] for i in range(N))
model_1.setObjective(ob_fn, GRB.MAXIMIZE)

In [167]:
model_1.setParam("OutputFlag", False)
model_1.optimize()

In [168]:
print("Objective Function Value: %f" % model_1.objVal)

Objective Function Value: 1.000000


In [169]:
import pandas as pd

In [170]:
output = []

for i in range(N):
    packed = int(round(x[i].X))
    output.append({
        'Box ID': i + 1,
        'Packed': 'Yes' if packed == 1 else 'No',
        'X': round(X[i].X, 2) if packed else '-',
        'Y': round(Y[i].X, 2) if packed else '-',
        'Z': round(Z[i].X, 2) if packed else '-',
        'Length': l[i],
        'Width': w[i],
        'Height': h[i]
    })

df = pd.DataFrame(output)
print(df.to_markdown(index=False))


|   Box ID | Packed   | X    | Y    | Z    |   Length |   Width |   Height |
|---------:|:---------|:-----|:-----|:-----|---------:|--------:|---------:|
|        1 | Yes      | 20.0 | 60.0 | 60.0 |       40 |      40 |       40 |
|        2 | No       | -    | -    | -    |       50 |      50 |       20 |
|        3 | No       | -    | -    | -    |       30 |      30 |       30 |
|        4 | No       | -    | -    | -    |       20 |      60 |       20 |
|        5 | No       | -    | -    | -    |       70 |      30 |       40 |


In [171]:
# a box can be placed in 6 ways
rotations = [
    (0, 1, 2),
    (0, 2, 1),
    (1, 0, 2),
    (1, 2, 0),
    (2, 0, 1),
    (2, 1, 0)
]
R = len(rotations)

box_dims = [
    (40, 40, 40),
    (50, 50, 20),
    (30, 30, 30),
    (20, 60, 20),
    (70, 30, 40)
]

In [172]:
def after_rotation(i, k):
    rot = rotations[k]
    new_dim = box_dims[i]
    return new_dim[rot[0]], new_dim[rot[1]], new_dim[rot[2]]
    

In [173]:
model_2 = Model("Model with rotations allowed")

In [174]:
y = model_2.addVars(N, vtype = GRB.BINARY)
r = model_2.addVars(N, R, vtype = GRB.BINARY, name = "rotations")
X_2 = model_2.addVars(N, vtype = GRB.CONTINUOUS, lb = 0)
Y_2 = model_2.addVars(N, vtype = GRB.CONTINUOUS, lb = 0)
Z_2 = model_2.addVars(N, vtype = GRB.CONTINUOUS, lb = 0)

a_2 = model_2.addVars(N, N, vtype = GRB.BINARY)
b_2 = model_2.addVars(N, N, vtype = GRB.BINARY)
c_2 = model_2.addVars(N, N, vtype = GRB.BINARY)

In [175]:
obj_fn_2 = sum(y[i] for i in range(N))
model_2.setObjective(obj_fn_2, GRB.MAXIMIZE)

In [176]:
model_2.addConstrs(sum(r[i, k] for k in range(R)) == y[i]
                  for i in range(N))

for i in range(N):
    for k in range(R):
        l_2, w_2, h_2 = after_rotation(i, k)
        model_2.addConstr(X_2[i] + l_2 * r[i, k] <= L)
        model_2.addConstr(Y_2[i] + w_2 * r[i, k] <= W)
        model_2.addConstr(Z_2[i] + h_2 * r[i, k] <= H)

model_2.addConstrs(a_2[i, i] == 0 for i in range(N))

model_2.addConstrs(b_2[i, i] == 0 for i in range(N))

model_2.addConstrs(c_2[i, i] == 0 for i in range(N))

model_2.addConstrs(a_2[i, j] + a_2[j, i] >= y[i] + y[j] - 1
                      for i in range(N) for j in range(N) if i != j)

model_2.addConstrs(b_2[i, j] + b_2[j, i] >= y[i] + y[j] - 1
                      for i in range(N) for j in range(N) if i != j)

model_2.addConstrs(c_2[i, j] + c_2[j, i] >= y[i] + y[j] - 1
                      for i in range(N) for j in range(N) if i != j)

model_2.addConstrs(a_2[i, j] + a_2[j, i] <= 1
                      for i in range(N) for j in range(N) if i != j)

model_2.addConstrs(b_2[i, j] + b_2[j, i] <= 1
                      for i in range(N) for j in range(N) if i != j)

model_2.addConstrs(c_2[i, j] + c_2[j, i] <= 1
                      for i in range(N) for j in range(N) if i != j)

M = L + W + H

for i in range(N):
    for j in range(N):
            for k in range(R):
                l_2, w_2, h_2 = after_rotation(i, k)
                model_2.addConstr(X_2[i] + l_2 * r[i, k] <= X_2[j] + M * (1 - a_2[i, j]))
                model_2.addConstr(X_2[i] + w_2 * r[i, k] <= Y_2[j] + M * (1 - b_2[i, j]))
                model_2.addConstr(X_2[i] + h_2 * r[i, k] <= Z_2[j] + M * (1 - c_2[i, j]))

model_2.addConstrs(y[i] >= p[i]*(y[j] - p[j])
                for i in range(N) for j in range(N))
#does not include the case if priority item is not feasible, but another items are feasible

#A better way to give priority is adding weights in objective function, N for priority ones and 1 for rest.

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,


In [181]:
# f = [1, 0, 0, 1, 0], 1 means no box can be placed above it
# if we want to include fragility constraint also, then just include this constraint 
# model_2.addConstrs(c_2[i, j]*f[i] == 0 for j in range(N) for i in range(N))

In [180]:
#if our objective is to maximize volume occupied, we just change objective function to:
#    sum(li*wi*hi*y[i] for i in range(N))

In [177]:
model_2.setParam("OutputFlag", False)
model_2.optimize()

In [178]:
print("Objective Function Value: %f" % model_2.objVal)

Objective Function Value: 1.000000


In [179]:
output = []

for i in range(N):
    packed = int(round(y[i].X))
    if packed:
        rot_k = [k for k in range(R) if r[i, k].X > 0.5][0]
        li, wi, hi = after_rotation(i, rot_k)
        output.append({
            'Box ID': i + 1,
            'Packed': 'Yes',
            'Rotation': rot_k + 1,
            'X': round(X_2[i].X, 2),
            'Y': round(Y_2[i].X, 2),
            'Z': round(Z_2[i].X, 2),
            'Length': li,
            'Width': wi,
            'Height': hi
        })
    else:
        output.append({
            'Box ID': i + 1,
            'Packed': 'No',
            'Rotation': '-',
            'X': '-', 'Y': '-', 'Z': '-',
            'Length': '-', 'Width': '-', 'Height': '-'
        })

df = pd.DataFrame(output)
print(df.to_markdown(index=False))


|   Box ID | Packed   | Rotation   | X   | Y    | Z    | Length   | Width   | Height   |
|---------:|:---------|:-----------|:----|:-----|:-----|:---------|:--------|:---------|
|        1 | No       | -          | -   | -    | -    | -        | -       | -        |
|        2 | No       | -          | -   | -    | -    | -        | -       | -        |
|        3 | No       | -          | -   | -    | -    | -        | -       | -        |
|        4 | No       | -          | -   | -    | -    | -        | -       | -        |
|        5 | Yes      | 2          | 0.0 | 60.0 | 70.0 | 70       | 40      | 30       |
